In [133]:
from os.path import join
import sys
[sys.path.append(i) for i in ['.', '..']]

import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

from config import FINAL_ANNOTATION_FILE_PATH, ORDINAL_COLS, NOMINAL_COLS, DATA_DIR
from evaluate.evaluate_counterspeech_diversity import extract_named_entities


In [114]:
df = pd.read_csv(FINAL_ANNOTATION_FILE_PATH)
df.fillna("n/a", inplace=True)
df.head(2)

/var/folders/5r/147qn_r949n270dn6qvhgnrw0000gn/T/ipykernel_26704/1642450606.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'n/a' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("n/a", inplace=True)


,theme,message,link,conspiracy theory,actor,strategy,goal,sheeples,secrecy,puppets,fear,hate,euphoria/excitement,authenticating,directive,rhetorical question,explictitly racist,explicitily antisemitic,explicitly misogynist,insults/offensive language,model_output,model,model_counter_speech,model_strategies,model_strategies_corrected,model_counter_speech_corrected,clarity,factual,alternative,narrative,critical_thinking,fear_id,fear_ack,conspiracy_usage,repetition_harmful,hate_speech_id,hate_speech_handling,conspiracy_handling,strategy_1,strategy_2,strategy_3,strategy_4,step_2,fear_id_message,fear_id_response,hate_speech_id_message,hate_speech_id_response
0,1.0,"""Wake up, sheeple! Biden's just a puppet for t...",1781568827693965722,QAnon,"Biden, Marxist elite",n/a,n/a,1,n/a,1,n/a,woke mob,n/a,0,1,n/a,n/a,n/a,n/a,n/a,"<XXX>\n\nAs a concerned citizen, I understand ...",llama3,"\n\nAs a concerned citizen, I understand your ...",,Strategy 1: Refute based on Fact-Checks\n\nStr...,"\n\nAs a concerned citizen, I understand your ...",2.0,2.0,n/a,n/a,3.0,0-1,2.0,1.0,4.0,0-0,n/a,n/a,1.0,1.0,-1.0,1.0,n/a,0.0,1.0,0.0,0.0
1,1.0,"""Liberals want to take away our #SecondAmendme...",1781568827693965722,QAnon,liberals,indirect: gun laws,take away our #SecondAmendmentRights and leave...,n/a,n/a,n/a,1.0,n/a,n/a,n/a,1,n/a,n/a,n/a,n/a,n/a,<XXX>\n\nThe Second Amendment is a vital part ...,llama3,\n\nThe Second Amendment is a vital part of ou...,\n\nHandled Step 2: No hate speech identified...,\n\nHandled Step 2: No hate speech identified...,\n\nThe Second Amendment is a vital part of ou...,4.0,4.0,4.0,n/a,2.0,0-0,n/a,1.0,2.0,0-0,n/a,n/a,1.0,1.0,-1.0,1.0,1.0,0.0,0.0,0.0,0.0


In [115]:
df = df[(df["model"]=="gpt4o") & (df["theme"]==2.0)]

In [116]:
len(df)

76

In [117]:
df_narrative = pd.read_csv(join(DATA_DIR, "annotation_ct_counter_speech_gpt4o_3.csv"), sep=";")
df_narrative.fillna("n/a", inplace=True)

/var/folders/5r/147qn_r949n270dn6qvhgnrw0000gn/T/ipykernel_26704/3409609122.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'n/a' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_narrative.fillna("n/a", inplace=True)


In [118]:
df_narrative.head(3)

,theme,message,link,conspiracy theory,actor,strategy,goal,sheeples,secrecy,puppets,fear,hate,euphoria/excitement,authenticating,directive,rhetorical question,explictitly racist,explicitily antisemitic,explicitly misogynist,insults/offensive language,counter_speech_gpt4o,model_strategies,model_counter_speech,clarity,narrative,factual,conspiracy_usage,repetition_harmful,hate_speech,hate_speech_handling,conspiracy_handling,comm_M
0,2.0,The @WorkersPartyGB fights the tyranny of #Bi...,https://x.com/georgegalloway/status/1809900994...,BigPharma,"Big Pharma, British state",compulsory medical intervention by the state d...,mass medication of the nation,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,<XXX>Understanding concerns about healthcare p...,- Step 2: No hate speech identified.\n- Step 3...,Understanding concerns about healthcare privat...,2.0,4.0,5.0,0,5.0,n/a,n/a,n/a,"geht nur son bissle am Punkt vorbei, weil die ..."
1,2.0,"""Big Pharma is pretty evil..."" - @LeilaniDowd...",https://x.com/ellymelly/status/180770219921425...,BigPharma,Big Pharma,They made us fat with their high-calorie fast ...,make profit,n/a,n/a,n/a,n/a,n/a,n/a,1,Watch it on @adhtvaus,n/a,n/a,n/a,n/a,n/a,<XXX>It's important to evaluate how companies ...,1. No hate speech was identified. \n2. The cla...,It's important to evaluate how companies opera...,2.0,2.0,n/a,0,5.0,n/a,n/a,n/a,Yoni Freedhoff ist nur leider nicht in der Ind...
2,2.0,"TRUMP TO INVESTIGATE BIG PHARMA FOR THE ""STUNN...",https://x.com/BoLoudon/status/1808187088455471349,BigPharma,Big Pharma,"AUTISM, INFERTILITY, ALLERGIES, & CHILD ILLNESS",defrauds American patients & taxpayers; make p...,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,<XXX>It's important to approach health concern...,- No hate speech was identified.\n- The claim ...,It's important to approach health concerns wit...,3.0,2.0,n/a,0,5.0,n/a,n/a,n/a,Fauci eher HIV-Forschung


In [119]:
REL_COLS = ["clarity", "narrative", "conspiracy_usage", "repetition_harmful"]

In [120]:
for c in ORDINAL_COLS:
    if c in df.columns:
        df[c] = df[c].map(lambda x: np.nan if x is None or x == "n/a" or float(x) == 0.0 or pd.isnull(x) else float(x))
    if c in df_narrative.columns:
        df_narrative[c] = df_narrative[c].map(lambda x: np.nan if x is None or x == "n/a" or float(x) == 0.0 or pd.isnull(x) else float(x))

In [121]:
df[["conspiracy_usage", "repetition_harmful"]] = df[["conspiracy_usage", "repetition_harmful"]].astype(float)


In [122]:
df_narrative[["conspiracy_usage", "repetition_harmful"]] = df_narrative[["conspiracy_usage", "repetition_harmful"]].astype(float)


In [123]:
ordinal_stats = df[REL_COLS].agg(
    ["mean", "median", "std", "min", "max"]
)

/Users/helenamihaljevic/.pyenv/versions/3.11.0/envs/twitter/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [124]:
df.narrative.notnull().sum()

0

In [125]:
ordinal_stats

,clarity,narrative,conspiracy_usage,repetition_harmful
mean,3.552632,NaN,0.105263,3.526316
median,4.000000,NaN,0.000000,3.500000
std,0.929441,NaN,0.308931,1.227178
min,1.000000,NaN,0.000000,1.000000
max,5.000000,NaN,1.000000,5.000000


In [126]:
ordinal_stats_narrative = df_narrative[REL_COLS].agg(
    ["mean", "median", "std", "min", "max"]
)
ordinal_stats_narrative

,clarity,narrative,conspiracy_usage,repetition_harmful
mean,3.533333,2.822222,0.065789,4.500000
median,4.000000,3.000000,0.000000,5.000000
std,0.759326,0.777200,0.249561,1.050114
min,2.000000,1.000000,0.000000,1.000000
max,5.000000,4.000000,1.000000,5.000000


In [127]:
df_narrative.narrative.notnull().sum()

45

In [128]:
100*df_narrative.narrative.notnull().sum()/len(df_narrative)

59.21052631578947